# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [2]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [3]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

NameError: name 'pd' is not defined

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

In [ ]:
pip install tabulate

In [6]:
import pandas as pd
from random import randint

1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [7]:
recipes = pd.read_csv('../02_pandas/recipes_sample.csv')

rand_ids = [randint(0, len(recipes)) for i in range(5)]

ids_mins = recipes['minutes'][rand_ids]
ids_mins = ids_mins.reset_index()
ids_mins.rename(columns={'index' : 'id'}, inplace=True) 
print(ids_mins.to_markdown(index=False))

|    id |   minutes |
|------:|----------:|
| 16957 |        39 |
|  7336 |        30 |
|  5405 |        35 |
| 10127 |        25 |
|  3628 |        40 |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [8]:
import xml.etree.ElementTree as ET

def get_steps(id):
        
    tree = ET.parse('../03_data_files/data/steps_sample.xml')
    steps = []
    for recipe in tree.getroot():
        if int(recipe[0].text) == id:
            for step in recipe[1]:
                steps.append(f"{len(steps) + 1}. {step.text}\n")
            return steps
            
    raise ValueError("Such id not found in steps_sample.xml")

def get_props(id):
    return recipes.loc[recipes.id == id]
    
def show_info(id):
    
    

    props = get_props(id)
    steps = get_steps(id)
    
    return \
    f'''
    {props['name'].values[0]}
    
    {"    ".join(steps)}
    ----------
    Автор: {props['contributor_id'].values[0]}
    Среднее время приготовления: {props['minutes'].values[0]} минут
    '''

print(show_info(170895))



    leeks and parsnips  sauteed or creamed
    
    1. clean the leeks and discard the dark green portions
    2. cut the leeks lengthwise then into one-inch pieces
    3. melt the butter in a medium skillet , med
    4. heat
    5. add the garlic and fry 'til fragrant
    6. add leeks and fry until the leeks are tender , about 6-minutes
    7. meanwhile , peel and chunk the parsnips into one-inch pieces
    8. place in a steaming basket and steam 'til they are as tender as you prefer
    9. i like them fork-tender
    10. drain parsnips and add to the skillet with the leeks
    11. add salt and pepper
    12. gently sautee together for 5-minutes
    13. at this point you can serve it , or continue on and cream it:
    14. in a jar with a screw top , add the half-n-half and arrowroot
    15. shake 'til blended
    16. turn heat to low under the leeks and parsnips
    17. pour in the arrowroot mixture , stirring gently as you pour
    18. if too thick , gradually add the water
    19. 

In [ ]:
assert (
    show_info(
        name="george s at the cove black bean soup",
        steps=[
            "clean the leeks and discard the dark green portions",
            "cut the leeks lengthwise then into one-inch pieces",
            "melt the butter in a medium skillet , med",
        ],
        minutes=90,
        author_id=35193,
    )
    == '"George S At The Cove Black Bean Soup"\n\n1. Clean the leeks and discard the dark green portions\n2. Cut the leeks lengthwise then into one-inch pieces\n3. Melt the butter in a medium skillet , med\n----------\nАвтор: 35193\nСреднее время приготовления: 90 минут\n'
)

## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [9]:
import re
for step in get_steps(25082):
    if re.findall(r"\d\d*\shour|hours|minute|minutes", step):
        print(step)

6. turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board

8. when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly

10. let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours

14. when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc

17. bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom



4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [22]:
recipes[recipes.description.str.contains(r'this[a-zA-Z0-9\s_]*\.\.\.,\s?but', na=False)]

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [40]:
for step in get_steps(72367):
    try:
        start = re.search(r'\d / \d', step).start()
        step = step[:start] + step[start:start+5].replace(" ", "") + step[start+5:]
    except AttributeError:
        pass
    print(step)
    

1. mix butter , flour , 1/3 c

2. sugar and 1-1/4 t

3. vanilla

4. press into greased 9" springform pan

5. mix cream cheese , 1/4 c

6. sugar , eggs and 1/2 t

7. vanilla beating until fluffy

8. pour over dough

9. combine apples , 1/3 c

10. sugar and cinnamon

11. arrange on top of cream cheese mixture and sprinkle with almonds

12. bake at 350 for 45-55 minutes , or until tester comes out clean



### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

words = []

for id in recipes['id']:
    for step in get_steps(id):
        for word in word_tokenize(step):
            if word.isalpha() and word.lower() not in words:
                words.append(word.lower())

# print(word_tokens)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\h\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(words)

7\. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

8\. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.
